In [1]:
import numpy as np
import pandas as pd
import re

In [40]:
df = pd.read_excel("../../../Preliminary_data/JSON full texts/VR_fulltext_fulldataset.xlsx")

#### Remove irrelevant articles

In [41]:
df.columns

Index(['id', 'publisher', 'subject', 'author', 'body', 'artdate', 'Month',
       'Year'],
      dtype='object')

In [42]:
#Remove all Spanish articles
spanish_outlets = ['El Nuevo Herald', 'El Diario La Prensa', 'Univision', 'AP Spanish Worldstream']
df = df[-df.publisher.isin(spanish_outlets)]

#Filter out Bloomberg show transcripts
df = df[-((df.publisher=='Bloomberg') & (df.subject.str.contains('Full Show')))]

#### Remove irrelevant text from articles

In [43]:
#Patterns at start of article
allarticle_header_regex = "Media: .*\nAuthor: (?:.*\n){1,10}Date: .*\n\n"
politico_share_regex = '.*\n{1,20}Follow Us\n'
politico_date_regex = '^.*\nBy.*\n\d\d/\d\d/\d\d\d\d \d\d:\d\d (?:AM|PM) EDT'

#Patterns at end of article
dow_regex = 'License this article from Dow Jones Reprint Service'

#Patterns in article
#search for line with only all caps and punctuation
fox_bold_regex = "\n[A-Z ',.-]+\n"

In [44]:
def regex_trim(rx_list,column, df=df, replace_value=""):
    '''Takes a list of regex patterns, and joins the patterns with an OR (|) separator. 
    Searches the specified column/df for the pattern and replaces it with value specified (default value-nothing)'''
    df[column] = df[column].replace(to_replace="|".join(rx_list), value=replace_value, regex=True)
    return df


In [45]:
remove_patterns = [allarticle_header_regex, 
                   dow_regex, 
                   politico_date_regex]

df = regex_trim(remove_patterns, "body")

#Remove ALL CAPS lines in Fox news articles
df["body"] = np.where(df.publisher=='Fox News', 
                           df.body.replace(to_replace=fox_bold_regex, value="", regex=True),
                           df.body)

#Remove irrelevant lines at start of Politico "playbook" articles
df["body"] = np.where(df.publisher=='Politico', 
                           df.body.replace(to_replace=politico_share_regex, value="", regex=True),
                           df.body)


#### Drop duplicates, null values

In [46]:
#Drop articles that have the same headline and date
df = df.drop_duplicates(["subject","artdate"])

#Drop articles that don't have a publisher listed
df = df[-df.publisher.isnull()]

In [47]:
df.head()

,id,publisher,subject,author,body,artdate,Month,Year
0,3622414,Associated Press,Trump signs order disbanding voter fraud commi...,"Colvin, Jill",All Rights Reserved ...,2018-01-04,1,2018
1,3622415,Wichita Eagle (KS),Pivot Point: GOP challengers will knock Kobach...,"Seminoff, Kirk",Pivot Point: GOP challengers will knock Kobach...,2018-01-04,1,2018
2,3622416,Associated Press,Trump disbands voter fraud commission amid fig...,"Colvin, Jill",1 of 4 DOCUMENTS ...,2018-01-04,1,2018
3,3622421,New York Daily News,Fraud of a panel KOd,"Sommerfeldt, Chris",Fraud of a panel KOd Media: New York Daily Ne...,2018-01-04,1,2018
4,3622425,The Kansas City Star,"We beat Trump on voter suppression this time, ...","Kander, Jason","We beat Trump on voter suppression this time, ...",2018-01-04,1,2018


In [48]:
df.to_excel('../preliminary_data/full_text_data_cleaning_result_VRfull.xlsx', index = False)

In [32]:
#Check for delimiter issues

#testdf = pd.read_excel('full_text_data_cleaning_result.xlsx')
#strcheck = testdf['id'].map(type) == int
#strcheck.value_counts()
#testdf['publisher'][testdf['publisher'].isnull()]

True    75393
Name: id, dtype: int64

In [ ]:
#Code for checking results of regex
#df[df.body_test.str.contains("\n[A-Z ',.-]+\n", regex=True)]

,id,Irrelevant,publisher,subject,Duplicate,author,body,artdate,Month,Year,"Notes: Green under Body (Good), Yellow under Subject (Duplicate), Red under Publisher (Spanish), Red under Subject (Some type of summary article of headlines/recent news)",body_test
31,19884512,NaN,The Arizona Republic,Debate has little substance for voters; Lack o...,NaN,By Yvonne Wingett Sanchez and Ronald J. Hansen...,Media: The Arizona Republic\nAuthor: By Yvonne...,2020-10-01,10,2020,NaN,Media: The Arizona Republic\nAuthor: By Yvonne...
122,19978780,NaN,Los Angeles Times,ELECTION 2020; Frustrations at the ballot box;...,NaN,Matt Stiles,Media: Los Angeles Times\nAuthor: Matt Stiles\...,2020-10-02,10,2020,NaN,Media: Los Angeles Times\nAuthor: Matt Stiles\...
125,19979151,NaN,The New York Times,"Get Your Culture, Inside and Out",NaN,NaN,Media: The New York Times\nAuthor: \nDate: 02 ...,2020-10-02,10,2020,NaN,Media: The New York Times\nAuthor: \nDate: 02 ...
147,19987524,NaN,Chicago Tribune,A graduated-rate income tax will be better for...,NaN,Eric Zorn,Media: Chicago Tribune\nAuthor: Eric Zorn\nDat...,2020-10-02,10,2020,NaN,Media: Chicago Tribune\nAuthor: Eric Zorn\nDat...
153,19989462,NaN,The Hill,"The Hill's Morning Report - Trump, first lady ...",NaN,Alexis Simendinger,Media: thehill\nAuthor: Alexis Simendinger\nDa...,2020-10-02,10,2020,NaN,Media: thehill\nAuthor: Alexis Simendinger\nDa...
...,...,...,...,...,...,...,...,...,...,...,...,...
5543,50050493,NaN,The Hill,The Hill's Morning Report - Trump finally conc...,NaN,Alexis Simendinger,Media: thehill\nAuthor: Alexis Simendinger\nDa...,2021-01-08,1,2021,NaN,Media: thehill\nAuthor: Alexis Simendinger\nDa...
5581,50050536,NaN,The Hill,The Hill's Morning Report - Presented by Faceb...,NaN,Alexis Simendinger,Media: thehill\nAuthor: Alexis Simendinger\nDa...,2021-01-06,1,2021,NaN,Media: thehill\nAuthor: Alexis Simendinger\nDa...
5582,50050812,NaN,The Hill,The Hill's Morning Report - Presented by Maste...,NaN,Alexis Simendinger,Media: thehill\nAuthor: Alexis Simendinger\nDa...,2020-12-18,12,2020,NaN,Media: thehill\nAuthor: Alexis Simendinger\nDa...
5583,50050938,NaN,The Hill,The Hill's Morning Report - Presented by Faceb...,NaN,Alexis Simendinger,Media: thehill\nAuthor: Alexis Simendinger\nDa...,2020-12-10,12,2020,NaN,Media: thehill\nAuthor: Alexis Simendinger\nDa...
